In [5]:
import sqlite3
import pandas as pd

In [18]:
cnx = sqlite3.connect("database.sqlite")

country_df = pd.read_sql_query("SELECT * FROM Country", cnx)
league_df = pd.read_sql_query("SELECT * FROM League", cnx)
match_df = pd.read_sql_query("SELECT * FROM Match", cnx)
player_df = pd.read_sql_query("SELECT * FROM Player", cnx)
player_att_df = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)
team_df = pd.read_sql_query("SELECT * FROM Team", cnx)
team_att_df = pd.read_sql_query("SELECT * FROM Team_Attributes", cnx

       id       name_x  country_id                    name_y
0       1      Belgium           1    Belgium Jupiler League
1    1729      England        1729    England Premier League
2    4769       France        4769            France Ligue 1
3    7809      Germany        7809     Germany 1. Bundesliga
4   10257        Italy       10257             Italy Serie A
5   13274  Netherlands       13274    Netherlands Eredivisie
6   15722       Poland       15722        Poland Ekstraklasa
7   17642     Portugal       17642  Portugal Liga ZON Sagres
8   19694     Scotland       19694   Scotland Premier League
9   21518        Spain       21518           Spain LIGA BBVA
10  24558  Switzerland       24558  Switzerland Super League


Index(['id', 'name_country', 'country_id', 'name_league'], dtype='object')
['id_x', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal', 'home_player_X1', 'home_player_X2', 'home_player_X3', 'home_player_X4', 'home_player_X5', 'home_player_X6', 'home_player_X7', 'home_player_X8', 'home_player_X9', 'home_player_X10', 'home_player_X11', 'away_player_X1', 'away_player_X2', 'away_player_X3', 'away_player_X4', 'away_player_X5', 'away_player_X6', 'away_player_X7', 'away_player_X8', 'away_player_X9', 'away_player_X10', 'away_player_X11', 'home_player_Y1', 'home_player_Y2', 'home_player_Y3', 'home_player_Y4', 'home_player_Y5', 'home_player_Y6', 'home_player_Y7', 'home_player_Y8', 'home_player_Y9', 'home_player_Y10', 'home_player_Y11', 'away_player_Y1', 'away_player_Y2', 'away_player_Y3', 'away_player_Y4', 'away_player_Y5', 'away_player_Y6', 'away_player_Y7', 'away_player_Y8', 'away_player_Y9', 'away_player_Y10', 'aw